# Amino acid mutation matrix and manual pruning phylogenetic tree

This notebook was used to 

1. parse Nextclade amino acid (AA) mutation information into an AA matrix compatible for visualization with [ggtree gheatmap](https://github.com/YuLab-SMU/ggtree/blob/master/R/gheatmap.R) with some code to check if sequence info was missing for certain mutations (i.e. Ns or gaps in the location of a mutation). Only the AA mutations present in the QC WTD sequences were considered and mutations in other sequences were dropped.
2. output a manually pruned tree for visualization using BioPython

The IQ-TREE tree was rooted on MN908947.3 and then MN908947.3 was pruned to better visualize branch lengths.

In [1]:
import pandas as pd

In [2]:
# read Nextclade GFF to get coordinates of genes
df_gff = pd.read_table('nextclade-sars-cov-2-dataset/genemap.gff', comment='#', header=None)
df_gff

,0,1,2,3,4,5,6,7,8
0,MN908947,GenBank,gene,266,13468,.,+,.,gene_name=ORF1a
1,MN908947,GenBank,gene,13468,21555,.,+,.,gene_name=ORF1b
2,MN908947,GenBank,gene,25393,26220,.,+,.,gene_name=ORF3a
3,MN908947,GenBank,gene,21563,25384,.,+,.,gene_name=S
4,MN908947,GenBank,gene,26245,26472,.,+,.,gene_name=E
5,MN908947,GenBank,gene,26523,27191,.,+,.,gene_name=M
6,MN908947,GenBank,gene,27202,27387,.,+,.,gene_name=ORF6
7,MN908947,GenBank,gene,27394,27759,.,+,.,gene_name=ORF7a
8,MN908947,GenBank,gene,27756,27887,.,+,.,gene_name=ORF7b
9,MN908947,GenBank,gene,27894,28259,.,+,.,gene_name=ORF8


In [3]:
df_gff = df_gff.loc[:,[3,4,8]]
df_gff.columns = ['start', 'end', 'gene']

In [4]:
df_gff.gene = df_gff.gene.str.replace('gene_name=', '').str.strip()

In [5]:
df_gff

,start,end,gene
0,266,13468,ORF1a
1,13468,21555,ORF1b
2,25393,26220,ORF3a
3,21563,25384,S
4,26245,26472,E
5,26523,27191,M
6,27202,27387,ORF6
7,27394,27759,ORF7a
8,27756,27887,ORF7b
9,27894,28259,ORF8


In [6]:
gff = {row.gene: (row.start, row.end) for row in df_gff.itertuples()}

In [7]:
gff

{'ORF1a': (266, 13468),
 'ORF1b': (13468, 21555),
 'ORF3a': (25393, 26220),
 'S': (21563, 25384),
 'E': (26245, 26472),
 'M': (26523, 27191),
 'ORF6': (27202, 27387),
 'ORF7a': (27394, 27759),
 'ORF7b': (27756, 27887),
 'ORF8': (27894, 28259),
 'N': (28274, 29533),
 'ORF9b': (28284, 28577)}

In [8]:
import re

In [9]:
regex_gene_aa_pos_simple = re.compile(r'(\w+):[a-zA-Z]+(\d+).*')

In [10]:
regex_gene_aa_pos_insertion = re.compile(r'(\w+):(\d+).*')

In [11]:
mut = 'ORF1a:2037:MRAS'
m = regex_gene_aa_pos_simple.match(mut)
if m:
    print(m.groups())
else:
    m = regex_gene_aa_pos_insertion.match(mut)
    if m:
        print(m.groups())

('ORF1a', '2037')


In [12]:
m = regex_gene_aa_pos_simple.match('ORF1a:N2038D')
if m:
    print(m.groups())

('ORF1a', '2038')


# Functions to get the nt position from AA mutation

From `{gene}:*{aa_position}*` to reference nucleotide position range for AA position.

For example `ORF1a:V23D` will map to a nucleotide from 332 to 335 (actual nt change T333A).

The `get_gene_nt_coords` will not work for frameshift mutations, however, but there's only one in the results.

In [13]:
from typing import Dict, Tuple, List

def split_mut(aamut: str) -> (str, int):
    m = regex_gene_aa_pos_simple.match(aamut)
    if m:
        gene, aa_pos = m.groups()
        return gene, int(aa_pos)
    else:
        m = regex_gene_aa_pos_insertion.match(aamut)
        if m:
            gene, aa_pos = m.groups()
            return gene, int(aa_pos)
        else:
            raise ValueError(f'Could not parse gene and AA position from {aamut}')

def get_gene_nt_coords(gff: Dict[str, Tuple[int, int]], gene: str, aa_pos: int) -> (int, int):
    if gene not in gff:
        raise ValueError(f'No gene {gene} in gene coordinate dict: {gff}')
    start, end = gff.get(gene)
    end_nt = aa_pos * 3 + start
    start_nt = end_nt - 3
    return start_nt, end_nt
    

In [14]:
aa_nt_muts = '''
ORF1a:V23D (T333A)
ORF1a:E159A (A741C)
ORF1a:T265I (C1059T)
ORF1a:M297V (A1154G)
ORF1a:H325Q (T1240A)
ORF1a:T619S (C2121G)
ORF1a:T708I (C2388T)
ORF1a:A735 (T2463TA [FRAMESHIFT])
ORF1a:D1289 (GA4130G [FRAMESHIFT])
ORF1a:V1290A (T4134C)
ORF1a:A1314V (C4206T)
ORF1a:A1809 (G5690GA [FRAMESHIFT])
ORF1a:L3116F (C9611T)
ORF1a:S3149F (CC9711TT)
ORF1a:K3353R (A10323G)
ORF1a:D3972V (A12180T)
ORF1a:V3976 (G12188GTT [FRAMESHIFT])
ORF1a:S3983F (C12213T)
ORF1a:C4326R (T13241C)
ORF1a:M4390T (T13434C)
ORF1b:R524C (C15037T)
ORF1b:V1271L (G17278T)
ORF1b:M1693I (G18546T)
ORF1b:P1727S (C18646T)
ORF1b:I2303V (A20374G)
ORF1b:A2469del (GTGC20870G [disruptive_inframe_deletion])
ORF1b:K2579R (A21203G)
S:F486L (T23020G)
S:N501T (A23064C)
S:D614G (A23403G)
S:S640F (C23481T)
S:S1003 (CA24566C [FRAMESHIFT])
S:M1237 (GTA25269G [FRAMESHIFT])
ORF3a:T12I (C25427T)
ORF3a:L219V (T26047G)
E:P71S (C26455T)
ORF8:D35Y (G27996T)
ORF8:E106* (G28209T)
N:P168S (C28775T)
N:S206P (T28889C)
N:T391I (C29445T)
ORF1a:S3983F (C12213T)
S:L1004S (T24573C)
S:A1070 (G24770GA [FRAMESHIFT])
S:A1070E (C24771A)
S:Q1071K (C24773A)
S:E1072K (G24776A)
E:P71S (C26455T)
ORF8:E106* (G28209T)
N:P168S (C28775T)
ORF1a:T265I (C1059T)
ORF1a:Y369 (TA1370T [FRAMESHIFT])
ORF1a:T708I (C2388T)
'''.strip().split('\n')

In [15]:
for aant_mut in aa_nt_muts:
    if ':' not in aant_mut:
        continue
    print(aant_mut)
    aamut, ntmut = aant_mut.split(' ', maxsplit=1)
    gene, aapos = split_mut(aamut)
    print(gene, aapos)
    print(get_gene_nt_coords(gff, gene, aapos))

ORF1a:V23D (T333A)
ORF1a 23
(332, 335)
ORF1a:E159A (A741C)
ORF1a 159
(740, 743)
ORF1a:T265I (C1059T)
ORF1a 265
(1058, 1061)
ORF1a:M297V (A1154G)
ORF1a 297
(1154, 1157)
ORF1a:H325Q (T1240A)
ORF1a 325
(1238, 1241)
ORF1a:T619S (C2121G)
ORF1a 619
(2120, 2123)
ORF1a:T708I (C2388T)
ORF1a 708
(2387, 2390)
ORF1a:A735 (T2463TA [FRAMESHIFT])
ORF1a 735
(2468, 2471)
ORF1a:D1289 (GA4130G [FRAMESHIFT])
ORF1a 1289
(4130, 4133)
ORF1a:V1290A (T4134C)
ORF1a 1290
(4133, 4136)
ORF1a:A1314V (C4206T)
ORF1a 1314
(4205, 4208)
ORF1a:A1809 (G5690GA [FRAMESHIFT])
ORF1a 1809
(5690, 5693)
ORF1a:L3116F (C9611T)
ORF1a 3116
(9611, 9614)
ORF1a:S3149F (CC9711TT)
ORF1a 3149
(9710, 9713)
ORF1a:K3353R (A10323G)
ORF1a 3353
(10322, 10325)
ORF1a:D3972V (A12180T)
ORF1a 3972
(12179, 12182)
ORF1a:V3976 (G12188GTT [FRAMESHIFT])
ORF1a 3976
(12191, 12194)
ORF1a:S3983F (C12213T)
ORF1a 3983
(12212, 12215)
ORF1a:C4326R (T13241C)
ORF1a 4326
(13241, 13244)
ORF1a:M4390T (T13434C)
ORF1a 4390
(13433, 13436)
ORF1b:R524C (C15037T)
ORF1b 524

In [16]:
!head nextclade/nextclade.tsv

index	seqName	clade	Nextclade_pango	partiallyAliased	clade_nextstrain	clade_who	clade_legacy	qc.overallScore	qc.overallStatus	totalSubstitutions	totalDeletions	totalInsertions	totalFrameShifts	totalMissing	totalNonACGTNs	totalAminoacidSubstitutions	totalAminoacidDeletions	totalAminoacidInsertions	totalUnknownAa	alignmentScore	alignmentStart	alignmentEnd	coverage	isReverseComplement	substitutions	deletions	insertions	frameShifts	aaSubstitutions	aaDeletions	aaInsertions	privateNucMutations.reversionSubstitutions	privateNucMutations.labeledSubstitutions	privateNucMutations.unlabeledSubstitutions	privateNucMutations.totalReversionSubstitutions	privateNucMutations.totalLabeledSubstitutions	privateNucMutations.totalUnlabeledSubstitutions	privateNucMutations.totalPrivateSubstitutions	missing	unknownAaRanges	nonACGTNs	qc.missingData.missingDataThreshold	qc.missingData.score	qc.missingData.status	qc.missingData.totalMissing	qc.mixedSites.mixedSitesThreshold	qc.mixedSites.score	qc.mixedSites.sta

In [17]:
df_nextclade = pd.read_table('nextclade/nextclade.tsv', dtype=str, index_col=1)

In [18]:
df_nextclade.columns

Index(['index', 'clade', 'Nextclade_pango', 'partiallyAliased',
       'clade_nextstrain', 'clade_who', 'clade_legacy', 'qc.overallScore',
       'qc.overallStatus', 'totalSubstitutions', 'totalDeletions',
       'totalInsertions', 'totalFrameShifts', 'totalMissing', 'totalNonACGTNs',
       'totalAminoacidSubstitutions', 'totalAminoacidDeletions',
       'totalAminoacidInsertions', 'totalUnknownAa', 'alignmentScore',
       'alignmentStart', 'alignmentEnd', 'coverage', 'isReverseComplement',
       'substitutions', 'deletions', 'insertions', 'frameShifts',
       'aaSubstitutions', 'aaDeletions', 'aaInsertions',
       'privateNucMutations.reversionSubstitutions',
       'privateNucMutations.labeledSubstitutions',
       'privateNucMutations.unlabeledSubstitutions',
       'privateNucMutations.totalReversionSubstitutions',
       'privateNucMutations.totalLabeledSubstitutions',
       'privateNucMutations.totalUnlabeledSubstitutions',
       'privateNucMutations.totalPrivateSubstituti

In [19]:
missing_series = df_nextclade.missing.str.split(',')

In [20]:
def range_str_to_int_tuple(nt_range: str) -> (int, int):
    if '-' in nt_range:
        start, end = nt_range.split('-')
        return int(start), int(end)
    else:
        return int(nt_range), int(nt_range)

In [21]:
from collections import defaultdict

In [22]:
import numpy as np

In [23]:
sample_missing_regions = {}
for sample, range_missing in missing_series.iteritems():
    if not isinstance(range_missing, list):
        print(f'sample {sample} has no missing regions')
        continue
    sample_missing_regions[sample] = [range_str_to_int_tuple(r) for r in range_missing]

sample OK034569.1 has no missing regions
sample OK020938.1 has no missing regions
sample OK035053.1 has no missing regions
sample OK035160.1 has no missing regions
sample OK089361.1 has no missing regions
sample OK119356.1 has no missing regions
sample OK177797.1 has no missing regions
sample OK287000.1 has no missing regions
sample OK286946.1 has no missing regions
sample OK352904.1 has no missing regions
sample OK463722.1 has no missing regions
sample OK443635.1 has no missing regions
sample OK403350.1 has no missing regions
sample OK443367.1 has no missing regions
sample OL360169.1 has no missing regions
sample OL564517.1 has no missing regions


/tmp/ipykernel_105558/1981736588.py:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for sample, range_missing in missing_series.iteritems():


In [24]:
import logging
from pathlib import Path
from typing import Set, List, Dict

import numpy as np
import pandas as pd
import typer


def parse(nextclade_csv: Path) -> pd.DataFrame:
    df_nextclade = pd.read_table(nextclade_csv, sep=';', dtype=str, index_col=1)
    aa_subs: pd.Series = df_nextclade['aaSubstitutions'].str.split(',')
    aa_dels: pd.Series = df_nextclade['aaDeletions'].str.split(',')
    aa_ins: pd.Series = df_nextclade['aaInsertions'].str.split(',')
    nt_subs: pd.Series = df_nextclade['substitutions'].str.split(',')
    sample_aas = sample_to_aa_mutations(aa_subs, aa_dels, aa_ins)
    sample_subs = sample_to_nt_mutations(nt_subs)
    samples = list(sample_aas.keys())
    unique_aas = get_sorted_aa_mutations(sample_aas)
    unique_subs = get_sorted_nt_subs(sample_subs)
    arr_aas = fill_aa_mutation_matrix(sample_aas, samples, unique_aas)
    arr_subs = fill_aa_mutation_matrix(sample_subs, samples, unique_subs)
    dfaa = pd.DataFrame(arr_aas, index=samples, columns=unique_aas)
    dfsubs = pd.DataFrame(arr_subs, index=samples, columns=unique_subs)
    return dfaa, dfsubs

def sample_to_nt_mutations(
        nt_subs: pd.Series
) -> Dict[str, Set[str]]:
    sample_subs = {}
    for sample, nt_sub in zip(nt_subs.index, nt_subs):
        subs = [] if not isinstance(nt_sub, list) else nt_sub
        sample_subs[sample] = set(subs)
    return sample_subs


def sample_to_aa_mutations(
        aa_subs: pd.Series,
        aa_dels: pd.Series
) -> Dict[str, Set[str]]:
    sample_aas = {}
    for sample, aa_sub, aa_del in zip(aa_subs.index, aa_subs, aa_dels):
        aas = [] if not isinstance(aa_sub, list) else aa_sub
        aad = [] if not isinstance(aa_del, list) else aa_del
        sample_aas[sample] = set(aas) | set(aad)
    return sample_aas


def fill_aa_mutation_matrix(
        sample_aas: Dict[str, Set[str]],
        samples: List[str],
        unique_aas: List[str]
) -> np.ndarray:
    """Fill AA mutation matrix with 1 when AA mutation present in sample"""
    arr_aas = np.zeros((len(sample_aas), len(unique_aas)), dtype='uint8')
    for i, sample in enumerate(samples):
        aas = sample_aas[sample]
        for j, aa in enumerate(unique_aas):
            if aa in aas:
                arr_aas[i, j] = 1
    return arr_aas


def get_sorted_aa_mutations(sample_aas: Dict[str, Set[str]]) -> List[str]:
    unique_aas = set()
    for aas in sample_aas.values():
        unique_aas |= aas
    unique_aas = list(unique_aas)
    unique_aas.sort()
    return unique_aas


def get_sorted_nt_subs(sample_subs: Dict[str, Set[str]]) -> List[str]:
    out = set()
    for subs in sample_subs.values():
        out |= subs
    out = list(out)
    out.sort()
    return out


def sample_to_aa_mutations(
        aa_subs: pd.Series,
        aa_dels: pd.Series,
        aa_insertions: pd.Series,
) -> Dict[str, Set[str]]:
    sample_aas = {}
    for sample, aa_sub, aa_del, aa_ins in zip(aa_subs.index, aa_subs, aa_dels, aa_insertions):
        aas = aa_sub if isinstance(aa_sub, list) else []
        aad = aa_del if isinstance(aa_del, list) else []
        aai = aa_ins if isinstance(aa_ins, list) else []
        sample_aas[sample] = set(aas) | set(aad) | set(aai)
    return sample_aas


In [25]:
dfaa, dfsubs = parse('nextclade/nextclade.csv')

In [26]:
dfaa.shape

(97, 119)

In [27]:
(dfaa.sum(axis=0) == 0)

E:N15S     False
E:R69I     False
M:I82T     False
N:D377Y    False
N:D63G     False
           ...  
S:S640F    False
S:T19R     False
S:T22I     False
S:T478K    False
S:V11A     False
Length: 119, dtype: bool

In [28]:
dfaa

,E:N15S,E:R69I,M:I82T,N:D377Y,N:D63G,N:G215C,N:R203M,N:S250A,ORF1a:A1306S,ORF1a:A2529V,...,S:G142D,S:H146Q,S:L452R,S:P681R,S:R158G,S:S640F,S:T19R,S:T22I,S:T478K,S:V11A
MZ839391.1,0,0,1,1,1,1,1,0,1,0,...,0,0,1,1,0,0,1,0,1,0
USA/CT-Yale-12907/2021,0,0,1,1,1,1,1,1,1,0,...,1,0,1,1,1,0,1,0,1,0
MZ778136.1,0,0,1,1,1,1,1,0,1,1,...,1,0,1,1,1,0,1,0,1,0
QC-4205,0,0,1,1,1,1,1,0,1,0,...,0,0,1,1,1,0,1,1,1,0
MZ801696.1,0,0,1,1,1,1,1,0,1,0,...,0,0,1,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OL662716.1,0,0,1,1,1,1,1,0,1,0,...,1,0,1,1,1,0,1,0,1,0
OL662092.1,0,0,1,1,1,1,1,0,1,0,...,1,0,1,1,1,0,1,0,1,0
OP930368.1,0,0,1,1,1,1,1,0,1,0,...,1,0,1,1,1,0,1,0,1,0
OL718610.1,0,0,1,1,1,1,1,0,1,0,...,1,0,1,1,1,0,1,0,1,0


In [29]:
dfaa

,E:N15S,E:R69I,M:I82T,N:D377Y,N:D63G,N:G215C,N:R203M,N:S250A,ORF1a:A1306S,ORF1a:A2529V,...,S:G142D,S:H146Q,S:L452R,S:P681R,S:R158G,S:S640F,S:T19R,S:T22I,S:T478K,S:V11A
MZ839391.1,0,0,1,1,1,1,1,0,1,0,...,0,0,1,1,0,0,1,0,1,0
USA/CT-Yale-12907/2021,0,0,1,1,1,1,1,1,1,0,...,1,0,1,1,1,0,1,0,1,0
MZ778136.1,0,0,1,1,1,1,1,0,1,1,...,1,0,1,1,1,0,1,0,1,0
QC-4205,0,0,1,1,1,1,1,0,1,0,...,0,0,1,1,1,0,1,1,1,0
MZ801696.1,0,0,1,1,1,1,1,0,1,0,...,0,0,1,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OL662716.1,0,0,1,1,1,1,1,0,1,0,...,1,0,1,1,1,0,1,0,1,0
OL662092.1,0,0,1,1,1,1,1,0,1,0,...,1,0,1,1,1,0,1,0,1,0
OP930368.1,0,0,1,1,1,1,1,0,1,0,...,1,0,1,1,1,0,1,0,1,0
OL718610.1,0,0,1,1,1,1,1,0,1,0,...,1,0,1,1,1,0,1,0,1,0


In [30]:
mb_deer_names = dfaa.index[dfaa.index.str.match(r'(AB|BC|NB|MB|QC|SK|ON)-')]

In [31]:

mb_deer_names

Index(['QC-4205', 'QC-4249', 'QC-4055', 'QC-4204'], dtype='object')

In [32]:
for x in mb_deer_names:
    print(x)

QC-4205
QC-4249
QC-4055
QC-4204


In [33]:
dfaa_on = dfaa.loc[mb_deer_names, :]

In [34]:
mut_mask = dfaa_on.sum() > 0

In [35]:
dfaa_sub = dfaa.loc[:, mut_mask]

In [36]:
dfaa_sub.index.name = 'sample'

In [37]:
df_gff

,start,end,gene
0,266,13468,ORF1a
1,13468,21555,ORF1b
2,25393,26220,ORF3a
3,21563,25384,S
4,26245,26472,E
5,26523,27191,M
6,27202,27387,ORF6
7,27394,27759,ORF7a
8,27756,27887,ORF7b
9,27894,28259,ORF8


In [38]:
df_gff.sort_values('start')

,start,end,gene
0,266,13468,ORF1a
1,13468,21555,ORF1b
3,21563,25384,S
2,25393,26220,ORF3a
4,26245,26472,E
5,26523,27191,M
6,27202,27387,ORF6
7,27394,27759,ORF7a
8,27756,27887,ORF7b
9,27894,28259,ORF8


In [39]:
ordered_genes = df_gff.sort_values('start').gene.to_list()

In [40]:
ordered_genes

['ORF1a',
 'ORF1b',
 'S',
 'ORF3a',
 'E',
 'M',
 'ORF6',
 'ORF7a',
 'ORF7b',
 'ORF8',
 'N',
 'ORF9b']

In [41]:
import re

In [42]:
re.sub(r'(\D+)?(\d+)\D+', r'\2', 'O:123:wha')

'123'

In [43]:
re.sub(r'(\D+)?(\d+)\D+', r'\2', 'O:A123T')

'123'

In [44]:
import re
from collections import defaultdict

def gene_ordered_mutations(muts) -> Dict[str, List[str]]:
    out = defaultdict(list)
    for m in muts:
        gene, mut = m.split(':', maxsplit=1)
        out[gene].append(mut)
    for k, v in out.items():
        v.sort(key=lambda x: int(re.sub(r'(\D+)?(\d+)\D+', r'\2', x)))
    return out

In [45]:
dfaa_sub.columns

Index(['M:I82T', 'N:D377Y', 'N:D63G', 'N:G215C', 'N:R203M', 'ORF1a:A1306S',
       'ORF1a:A2554V', 'ORF1a:H2125Y', 'ORF1a:L1853F', 'ORF1a:L3116F',
       'ORF1a:L3606F', 'ORF1a:M85-', 'ORF1a:P2046L', 'ORF1a:P2287S',
       'ORF1a:Q1784H', 'ORF1a:S2242F', 'ORF1a:T2823I', 'ORF1a:T3255I',
       'ORF1a:T3646A', 'ORF1a:V1143F', 'ORF1a:V2930L', 'ORF1b:A1918V',
       'ORF1b:F183V', 'ORF1b:G662S', 'ORF1b:H1087Y', 'ORF1b:I1498V',
       'ORF1b:P1000L', 'ORF1b:P314L', 'ORF1b:S69F', 'ORF3a:R134H',
       'ORF3a:S26L', 'ORF3a:T12I', 'ORF7a:T120I', 'ORF7a:T39I', 'ORF7a:V82A',
       'ORF7b:L4F', 'ORF7b:T40I', 'ORF8:A65-', 'ORF8:C61-', 'ORF8:D119-',
       'ORF8:D63-', 'ORF8:E64-', 'ORF8:F120-', 'ORF8:G66-', 'ORF8:K68*',
       'ORF8:L60-', 'ORF8:L60F', 'ORF8:S67F', 'ORF8:V62-', 'ORF9b:T60A',
       'S:A27V', 'S:D614G', 'S:D950N', 'S:E156-', 'S:F157-', 'S:G1085R',
       'S:L452R', 'S:P681R', 'S:R158G', 'S:T19R', 'S:T22I', 'S:T478K'],
      dtype='object')

In [59]:
gmuts = gene_ordered_mutations(dfaa_sub.columns)

In [70]:
ordered_gmuts = [f'{g}:{x}' for g in ordered_genes for x in gmuts[g]]

In [71]:
for x in ordered_gmuts:
    print(x)

ORF1a:M85-
ORF1a:V1143F
ORF1a:A1306S
ORF1a:Q1784H
ORF1a:L1853F
ORF1a:P2046L
ORF1a:H2125Y
ORF1a:S2242F
ORF1a:P2287S
ORF1a:A2554V
ORF1a:T2823I
ORF1a:V2930L
ORF1a:L3116F
ORF1a:T3255I
ORF1a:L3606F
ORF1a:T3646A
ORF1b:S69F
ORF1b:F183V
ORF1b:P314L
ORF1b:G662S
ORF1b:P1000L
ORF1b:H1087Y
ORF1b:I1498V
ORF1b:A1918V
S:T19R
S:T22I
S:A27V
S:E156-
S:F157-
S:R158G
S:L452R
S:T478K
S:D614G
S:P681R
S:D950N
S:G1085R
ORF3a:T12I
ORF3a:S26L
ORF3a:R134H
M:I82T
ORF7a:T39I
ORF7a:V82A
ORF7a:T120I
ORF7b:L4F
ORF7b:T40I
ORF8:L60-
ORF8:L60F
ORF8:C61-
ORF8:V62-
ORF8:D63-
ORF8:E64-
ORF8:A65-
ORF8:G66-
ORF8:S67F
ORF8:K68*
ORF8:D119-
ORF8:F120-
N:D63G
N:R203M
N:G215C
N:D377Y
ORF9b:T60A


In [72]:
dfaa_sub[ordered_gmuts].sum(axis=0)

ORF1a:M85-       1
ORF1a:V1143F     3
ORF1a:A1306S    97
ORF1a:Q1784H    14
ORF1a:L1853F     1
                ..
N:D63G          97
N:R203M         97
N:G215C         97
N:D377Y         96
ORF9b:T60A      97
Length: 62, dtype: int64

In [73]:
dfaa_sub[ordered_gmuts].sum(axis=0)

ORF1a:M85-       1
ORF1a:V1143F     3
ORF1a:A1306S    97
ORF1a:Q1784H    14
ORF1a:L1853F     1
                ..
N:D63G          97
N:R203M         97
N:G215C         97
N:D377Y         96
ORF9b:T60A      97
Length: 62, dtype: int64

In [74]:
samples_to_show = mb_deer_names

In [75]:
ordered_gmuts

['ORF1a:M85-',
 'ORF1a:V1143F',
 'ORF1a:A1306S',
 'ORF1a:Q1784H',
 'ORF1a:L1853F',
 'ORF1a:P2046L',
 'ORF1a:H2125Y',
 'ORF1a:S2242F',
 'ORF1a:P2287S',
 'ORF1a:A2554V',
 'ORF1a:T2823I',
 'ORF1a:V2930L',
 'ORF1a:L3116F',
 'ORF1a:T3255I',
 'ORF1a:L3606F',
 'ORF1a:T3646A',
 'ORF1b:S69F',
 'ORF1b:F183V',
 'ORF1b:P314L',
 'ORF1b:G662S',
 'ORF1b:P1000L',
 'ORF1b:H1087Y',
 'ORF1b:I1498V',
 'ORF1b:A1918V',
 'S:T19R',
 'S:T22I',
 'S:A27V',
 'S:E156-',
 'S:F157-',
 'S:R158G',
 'S:L452R',
 'S:T478K',
 'S:D614G',
 'S:P681R',
 'S:D950N',
 'S:G1085R',
 'ORF3a:T12I',
 'ORF3a:S26L',
 'ORF3a:R134H',
 'M:I82T',
 'ORF7a:T39I',
 'ORF7a:V82A',
 'ORF7a:T120I',
 'ORF7b:L4F',
 'ORF7b:T40I',
 'ORF8:L60-',
 'ORF8:L60F',
 'ORF8:C61-',
 'ORF8:V62-',
 'ORF8:D63-',
 'ORF8:E64-',
 'ORF8:A65-',
 'ORF8:G66-',
 'ORF8:S67F',
 'ORF8:K68*',
 'ORF8:D119-',
 'ORF8:F120-',
 'N:D63G',
 'N:R203M',
 'N:G215C',
 'N:D377Y',
 'ORF9b:T60A']

In [76]:
dfaa_sub.loc[samples_to_show,:][ordered_gmuts]

,ORF1a:M85-,ORF1a:V1143F,ORF1a:A1306S,ORF1a:Q1784H,ORF1a:L1853F,ORF1a:P2046L,ORF1a:H2125Y,ORF1a:S2242F,ORF1a:P2287S,ORF1a:A2554V,...,ORF8:G66-,ORF8:S67F,ORF8:K68*,ORF8:D119-,ORF8:F120-,N:D63G,N:R203M,N:G215C,N:D377Y,ORF9b:T60A
QC-4205,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
QC-4249,0,0,1,1,0,1,1,0,1,1,...,0,0,0,1,1,1,1,1,1,1
QC-4055,0,1,1,1,0,1,1,0,1,1,...,0,0,0,1,1,1,1,1,1,1
QC-4204,1,0,1,1,0,0,1,1,0,1,...,0,0,0,0,0,1,1,1,1,1


In [77]:
muts_in_gt50 = dfaa_sub.loc[samples_to_show,:][ordered_gmuts].sum(axis=0) >= 1

In [78]:
muts_in_gt50

ORF1a:M85-      True
ORF1a:V1143F    True
ORF1a:A1306S    True
ORF1a:Q1784H    True
ORF1a:L1853F    True
                ... 
N:D63G          True
N:R203M         True
N:G215C         True
N:D377Y         True
ORF9b:T60A      True
Length: 62, dtype: bool

In [79]:
len(ordered_gmuts)

62

## Matrix of AA mutation binary absence/presence to gene name if present, dash if absent

AA mutation sites with no sequencing coverage (Ns or gaps) were filled in with `*No Coverage`

In [80]:
for col in dfaa_sub:
    gene,_ = col.split(':', maxsplit=1)
    series = dfaa_sub[col]
    series[series == 0] = '-'
    series[series == 1] = gene

In [81]:
for sample, missing_ranges in sample_missing_regions.items():    
    for gmut in ordered_gmuts:
        gene, aapos = split_mut(gmut)
        start, end = get_gene_nt_coords(gff, gene, aapos)
        for missing_start, missing_end in missing_ranges:
            if missing_end >= start >= missing_start or missing_end >= end >= missing_start:
                print(gmut)
                print(start, end)
                print(missing_start, missing_end)
                print('='*80)
                dfaa_sub.loc[sample, gmut] = '*No Coverage'

S:E156-
22028 22031
21717 22261
S:F157-
22031 22034
21717 22261
S:R158G
22034 22037
21717 22261
ORF1a:S2242F
6989 6992
6860 7084
S:E156-
22028 22031
21744 22261
S:F157-
22031 22034
21744 22261
S:R158G
22034 22037
21744 22261
ORF1a:S2242F
6989 6992
6847 7058
S:E156-
22028 22031
21717 22262
S:F157-
22031 22034
21717 22262
S:R158G
22034 22037
21717 22262
ORF1a:S2242F
6989 6992
6847 7034
S:E156-
22028 22031
21717 22290
S:F157-
22031 22034
21717 22290
S:R158G
22034 22037
21717 22290
N:D377Y
29402 29405
29357 29510
ORF1a:S2242F
6989 6992
6847 7034
S:E156-
22028 22031
21717 22290
S:F157-
22031 22034
21717 22290
S:R158G
22034 22037
21717 22290
ORF1a:S2242F
6989 6992
6847 7058
ORF1a:S2242F
6989 6992
6847 7034
ORF1b:A1918V
19219 19222
18980 19570
S:E156-
22028 22031
21717 22290
S:F157-
22031 22034
21717 22290
S:R158G
22034 22037
21717 22290
ORF1a:V1143F
3692 3695
3506 4087
ORF1a:P2046L
6401 6404
6237 6495
ORF1a:P2287S
7124 7127
7073 7332
ORF1a:T2823I
8732 8735
8636 8940
ORF1b:I1498V
17959 17962


In [82]:
dfaa_sub

,M:I82T,N:D377Y,N:D63G,N:G215C,N:R203M,ORF1a:A1306S,ORF1a:A2554V,ORF1a:H2125Y,ORF1a:L1853F,ORF1a:L3116F,...,S:D950N,S:E156-,S:F157-,S:G1085R,S:L452R,S:P681R,S:R158G,S:T19R,S:T22I,S:T478K
sample,,,,,,,,,,,,,,,,,,,,,
MZ839391.1,M,N,N,N,N,ORF1a,ORF1a,ORF1a,-,-,...,S,*No Coverage,*No Coverage,S,S,S,*No Coverage,S,-,S
USA/CT-Yale-12907/2021,M,N,N,N,N,ORF1a,ORF1a,ORF1a,-,-,...,S,S,S,S,S,S,S,S,-,S
MZ778136.1,M,N,N,N,N,ORF1a,ORF1a,ORF1a,-,-,...,S,S,S,S,S,S,S,S,-,S
QC-4205,M,N,N,N,N,ORF1a,ORF1a,ORF1a,ORF1a,ORF1a,...,S,S,S,S,S,S,S,S,S,S
MZ801696.1,M,N,N,N,N,ORF1a,ORF1a,ORF1a,-,-,...,S,S,S,S,S,S,S,S,-,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OL662716.1,M,N,N,N,N,ORF1a,ORF1a,ORF1a,-,-,...,S,S,S,S,S,S,S,S,-,S
OL662092.1,M,N,N,N,N,ORF1a,ORF1a,ORF1a,-,-,...,S,S,S,S,S,S,S,S,-,S
OP930368.1,M,N,N,N,N,ORF1a,ORF1a,ORF1a,-,-,...,S,S,S,S,S,S,S,S,-,S


In [83]:
dfaa_sub.loc[:, ordered_gmuts].to_csv('aa-matrix.tsv', sep='\t')

In [84]:
import numpy as np

In [85]:
list(np.unique(dfaa_sub.values))

['*No Coverage',
 '-',
 'M',
 'N',
 'ORF1a',
 'ORF1b',
 'ORF3a',
 'ORF7a',
 'ORF7b',
 'ORF8',
 'ORF9b',
 'S']

In [86]:
dfaa_sub[ordered_gmuts].to_csv('aa-matrix.tsv', sep='\t')

In [87]:
dfaa_sub = dfaa_sub[ordered_gmuts]

In [88]:
mut_pattern_samples = defaultdict(list)
for sample, mut_pattern in dfaa_sub.apply(lambda x: '|'.join(x), axis=1).items():
    mut_pattern_samples[mut_pattern].append(sample)

Output basic table of sample names and number of collapsed taxa represented by a sample.

In [89]:
nr_samples = []
with open('aa-mut-pattern-collapsed-count.tsv', 'w') as fout:
    for samples in mut_pattern_samples.values():
        sample = samples[0]
        nr_samples.append(sample)
        fout.write(f'{sample}\t{len(samples) - 1}\n')

In [90]:
len(nr_samples)

21

In [91]:
dfaa_sub.loc[nr_samples,:].to_csv('aa-matrix-collapsed-nr-samples.tsv', sep='\t')

In [108]:
from Bio import Phylo

In [109]:
subtree = Phylo.read('iqtree-2023-03-29-QC-WTD-and-related-GISAID-NCBI.treefile', 'newick')

In [110]:
subtree.count_terminals()

98

In [111]:
subtree.root_with_outgroup('MN908947.3')

In [113]:
subtree.prune('MN908947.3')

Clade()

In [114]:
subtree.count_terminals()

97

In [115]:
Phylo.write(subtree, 'tree-full.newick', 'newick', format_branch_length="%f")

1

In [95]:
name_node = {n.name: n for n in subtree.get_terminals()}

In [96]:
nr_samples

['MZ839391.1',
 'USA/CT-Yale-12907/2021',
 'MZ778136.1',
 'QC-4205',
 'MZ831164.1',
 'QC-4249',
 'Canada/QC-L00415260001B/2021',
 'MZ831131.1',
 'QC-4055',
 'QC-4204',
 'OK035071.1',
 'OK176181.1',
 'OK089805.1',
 'OK179639.1',
 'OL545998.1',
 'OL564517.1',
 'OL662701.1',
 'OL662700.1',
 'OL662716.1',
 'OL718610.1',
 'OL846289.1']

In [97]:
samples_to_show = set(nr_samples) | {'MN908947.3'}

In [98]:
samples_to_show

{'Canada/QC-L00415260001B/2021',
 'MN908947.3',
 'MZ778136.1',
 'MZ831131.1',
 'MZ831164.1',
 'MZ839391.1',
 'OK035071.1',
 'OK089805.1',
 'OK176181.1',
 'OK179639.1',
 'OL545998.1',
 'OL564517.1',
 'OL662700.1',
 'OL662701.1',
 'OL662716.1',
 'OL718610.1',
 'OL846289.1',
 'QC-4055',
 'QC-4204',
 'QC-4205',
 'QC-4249',
 'USA/CT-Yale-12907/2021'}

In [99]:
subtree.count_terminals()

98

In [100]:
for name, node in name_node.items():
    if name in samples_to_show:
        continue
    subtree.prune(node)

In [101]:
subtree.count_terminals()

22

In [103]:
subtree.root_with_outgroup('MN908947.3')

In [105]:
subtree.prune('MN908947.3')

Clade()

In [106]:
subtree.count_terminals()

21

In [107]:
Phylo.write(subtree, 'tree-pruned.newick', 'newick', format_branch_length="%f")

1